# Webscrape Righmove
Goal: Collect data on all the listings in nottingham and create a color coordinated map of rent accross the city


In [47]:
#Import Libraries
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager


from selenium.webdriver.common.by import By

import pandas as pd
import numpy as np
import time

from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.keys import Keys

# 1. Initiate the driver

In [68]:

#open url
url = 'https://www.rightmove.co.uk/'

#create driver
driver = webdriver.Edge(EdgeChromiumDriverManager().install())
driver.get(url)

time.sleep(2)


In [69]:
#Allow Cookies
driver.find_element(By.XPATH,'//*[@id="onetrust-accept-btn-handler"]').click()

In [70]:
# navigate to houses in nottingham
city = 'Nottingham'
driver.find_element(By.NAME,"typeAheadInputField").send_keys(city,Keys.RETURN)


In [71]:
#find properties btn
driver.find_element(By.CSS_SELECTOR,'button.touchsearch-button.touchsearch-primarybutton').click()

### Logic and Understanding

I want data on all houses/flats for sale 

I want to scrape the data change page and repeat

**The logic works !!!**
- Change the page then use driver.page_source to download the source code

# 2. Scrape the html and change page
- Dynamic Webscraping


#### Confirm it works

In [73]:
#build logic

def parse_website():

    #Get Number of pages
    total_pages = driver.find_element(By.XPATH,'//*[@id="l-container"]/div[3]/div/div/div/div[2]/span[3]').text

    #store the pages html
    broth = []

    for i in range(0,int(total_pages)):    

        time.sleep(2)

        #get html
        page_i = driver.page_source

        #create soup (turn html into a parsable text)
        soup = BeautifulSoup(page_i, 'html.parser')
        broth.append(soup)

        #explicit wait -  wait for the next page to load
        wait =  WebDriverWait(driver,10)
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR,'button.pagination-button.pagination-direction.pagination-direction--next')))

        #click next page btn
        driver.find_element(By.CSS_SELECTOR,'button.pagination-button.pagination-direction.pagination-direction--next').click()

    return broth
    
broth = parse_website()

In [72]:
driver.page_source

'<html lang="en-GB" class="is-not-modern property-for-sale channel--buy sticky-filters"><head>\n    <meta charset="utf-8">\n    <title>Properties For Sale in Nottingham | Rightmove</title>\n    <meta http-equiv="X-UA-Compatible" content="IE=Edge">\n    <meta name="viewport" content="width=device-width, shrink-to-fit=no, initial-scale=1.0, user-scalable=no">\n    <meta name="format-detection" content="telephone=no">\n    <meta name="HandheldFriendly" content="True">\n    <meta name="description" content="Flats &amp; Houses For Sale in Nottingham - Find properties with Rightmove - the UK\'s largest selection of properties.">\n    <meta name="title" content="">\n    <meta name="referrer" content="origin-when-cross-origin"><link rel="preconnect" href="https://media.rightmove.co.uk:443" crossorigin="">\n    <link rel="preconnect" href="//product.rightmove.co.uk" crossorigin=""><link rel="shortcut icon" href="/pvw/images/favicons/rebranded/favicon.ico"><link rel="apple-touch-icon" sizes="72x

In [61]:
len(broth)

42

# 3. Parse the HTML
- Extract data 
- format the data


###  Extract features into dataframe

In [74]:

def create_df(broth):

    address = []
    house_type = []
    bedrooms = []
    bathrooms = []

    #create dataframe to hold the 
    data = pd.DataFrame(columns = ['address','house_type','bedrooms','bathrooms'])

    for i,soup in enumerate(broth):

        #get the property info objects from each page
        prop_info = soup.find_all('div',{'class':'property-information'})

        #Loop through each property info object and gater the addy + info
        for x in prop_info:

            #go up one node to get the address
            address.append(x.previous_sibling.text)

            #descend 1 node to get to the house type, number of bathrooms and beds
            house_type.append(x.find_all('span')[0].text)


            #When ther isnt a value for beds of bathrooms append a nan value
            try:
                bedrooms.append(x.find_all('span')[1].text[0])
            except:
                bedrooms.append(np.nan)

            try:
                bathrooms.append(x.find_all('span')[2].text[0])
            except:
                bathrooms.append(np.nan)
                
                
        #format the guide price
    guide_prices_str = [x.text  for i in broth for x in i.find_all('div',{'class':'propertyCard-priceValue'})  ]
    pp1 = [i.replace('£','') for i in guide_prices_str]
    guide_prices = [i.replace(',','') for i in pp1]


    #sort out POA values
    guide_prices = [np.nan if x in ['POA','Coming Soon','Sale by Tender'] else x for x in guide_prices]
    # Convert the values in the list to float
    guide_prices = [float(value) for value in guide_prices]

    # add data to dataframe
    data['guide_price'] = guide_prices
    data['address'],data['house_type'],data['bedrooms'],data['bathrooms']=[address,house_type,bedrooms,bathrooms]

    return data

data = create_df(broth)

In [75]:
data.head()

,address,house_type,bedrooms,bathrooms,guide_price
0,"Queens Road, Nottingham, Nottinghamshire, NG2",Flat,2,2,145000.0
1,"Derby Rd, Nottingham",Not Specified,2,2,3499999.0
2,"Boden House, West Gate, Long Eaton, NG10 1EG",Block of Apartments,NaN,NaN,3000000.0
3,"Magdala Road, Mapperley Park",Detached,8,8,2550000.0
4,"Meadow Lane, Nottingham",Land for sale,NaN,NaN,NaN


### This is Sexual!!

In [76]:
#Export the table 
data.to_csv('rightmove_data',index = False)

### Data set is now created !
- Follow up is to train a predictive model 
- Use geopandas to vizualize price and location